In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

<h2>登录并打开查询页面

In [2]:
browser = webdriver.Chrome()
browser.maximize_window()  # 最大化窗口
wait = WebDriverWait(browser, 10) # 等待加载10s
url = 'http://www.bhi.com.cn/Login/login.aspx?prev=http://projectinfo.bhi.com.cn/Projects/ProjectNList.aspx'
browser.get(url)
browser.set_page_load_timeout(120)

In [ ]:
# 输入用户名、密码和验证码

In [3]:
submit = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="login"]')))
submit.click()

In [4]:
input = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="keyword"]')))
input.clear()
input.send_keys("电梯")
check = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="industry"]/li[1]/label[7]/input')))
check.click()
check = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="industry"]/li[1]/label[8]/input')))
check.click()
check = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="industry"]/li[1]/label[10]/input')))
check.click()
check = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="form1"]/div[9]/div[1]/input[1]')))
check.click()

<h2>添加项目数据至database中

In [6]:
def turn_page(browser):
    curHandle = browser.current_window_handle #获取当前窗口聚丙
    allHandle = browser.window_handles #获取所有聚丙　　　　
    for h in allHandle:
        if h != curHandle:
            browser.switch_to.window(h) #切换聚丙，到新弹出的窗口
            break

In [7]:
def close_turn_page(browser):
    curHandle = browser.current_window_handle #获取当前窗口聚丙
    allHandle = browser.window_handles #获取所有聚丙　　　　
    for h in allHandle:
        if h != curHandle:
            browser.close()
            browser.switch_to.window(h) #切换聚丙，到新弹出的窗口
            break

In [8]:
def basic_info(project_page):
    title = project_page.find('h1').get_text()
    title = title.replace('\n','').replace(' ','')

    info = [x.get_text() for x in project_page.find_all('td')]
    info = [text.replace('\n','').replace(' ','') for text in info]
    region = info[2]
    post_date = info[4]
    industry = info[6]
    con_period = info[8] #construction_period
    investment = info[10]
    progress = info[12]
    equipments = info[20]
    description = info[24]
    return info,title,region,post_date,industry,con_period,investment,progress,equipments,description

In [9]:
def bodi_info(info):
    is_bo = False
    is_di = False
    is_re = False
    if '业主方及其联系方式' in info:
        is_bo = True
    if '设计单位及其联系方式' in info:
        is_di = True
    if '可研单位及其联系方式' in info:
        is_re = True

    if is_bo and is_di:
        bo_ind = info.index('业主方及其联系方式')
        di_ind = info.index('设计单位及其联系方式')
        bo_name = info[bo_ind+2].replace('近期参与项目','')
        bo_contact_list = info[bo_ind+7:di_ind]
        bo_contact = ",".join(bo_contact_list)
        di_name = info[di_ind+2].replace('近期参与项目','')
        if is_re:
            re_ind = info.index('可研单位及其联系方式')
            di_contact_list = info[di_ind+7:re_ind]
        else:
            di_contact_list = info[di_ind+7:]
        di_contact = ",".join(di_contact_list)
    elif is_bo and not is_di:
        bo_ind = info.index('业主方及其联系方式')
        bo_name = info[bo_ind+2].replace('近期参与项目','')    
        if is_re:
            re_ind = info.index('可研单位及其联系方式')
            bo_contact_list = info[bo_ind+7:re_ind]
        else:
            bo_contact_list = info[bo_ind+7:]
        bo_contact = ",".join(bo_contact_list)
        di_name = ''
        di_contact = ''
    else:
        pass
    return bo_name,bo_contact,di_name,di_contact

In [26]:
cols = ["title","region","post_date","industry","con_period","investment","progress","equipments","description"]
cols += ["bo_name","bo_contact","di_name","di_contact"]
database = pd.DataFrame(columns = cols)

In [27]:
for j in range(0,78):
    i = 0
    while i < 30:
        check = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="list"]/tbody/tr[%d]/td[1]/input'%(i+2))))
        check.click()
        submit = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="form1"]/div[11]/div[2]/div[1]/a[1]')))
        submit.click()
        time.sleep(np.random.rand()+2)
        turn_page(browser)
        project_content = browser.page_source.encode('utf-8')
        project_page = BeautifulSoup(project_content, 'lxml')
        info,title,region,post_date,industry,con_period,investment,progress,equipments,description = basic_info(project_page)
        bo_name,bo_contact,di_name,di_contact = bodi_info(info)
        database.loc[i+30*j,:] = [title,region,post_date,industry,con_period,investment,progress,equipments,description,bo_name,bo_contact,di_name,di_contact]
        i = i + 1
        close_turn_page(browser)
        time.sleep(np.random.rand()+2)
        check = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="list"]/tbody/tr[%d]/td[1]/input'%(i+1))))
        check.click()
    nextpage = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="next"]')))
    nextpage.click()
    time.sleep(np.random.rand()+10)

AttributeError: 'NoneType' object has no attribute 'get_text'

In [29]:
len(database)

300

In [30]:
database.to_csv("database_0906.csv")

In [32]:
database.to_excel("database_0906.xlsx")

In [33]:
browser.quit()